In [4]:
import torch
import sys
sys.path.append("../")
from shared_utils.load import get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text
from early_exit.util import get_model, load_model
from early_exit.patching import replace_attention_layers, set_transformer_early_exit_mode

model_path = "../models/early_exit_20250811_layers_5_big/step_2000"
base_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
config = "../config_deepseek.yaml"
device = "cuda"

In [5]:
tokenizer = get_tokenizer(base_model)
config = configs_from_yaml(config, tokenizer.eos_token_id)

base_model = get_model(base_model, config['model'], device)
model = replace_attention_layers(base_model, config['lora'], device)
model = load_model(model, model_path)

print(f"Model loaded w exitable layers: {model.exitable_layer_idxs}")

address this hack!
trainable params: 2,179,072 || all params: 1,802,890,757 || trainable%: 0.1209
Model loaded w exitable layers: tensor([ 5., 10., 15., 20., 25., inf])


In [6]:
set_transformer_early_exit_mode(model, 'free_generate')

In [7]:
prompt = "Explain the concept of recursion in programming."
system_prompt = "You are a helpful programming tutor."
prefiller = ""

In [8]:
with torch.no_grad():
    try:
        free_generate_response, exit_info = generate_text(
            model=model,
            prompt=prompt,
            system_prompt=system_prompt,
            prefiller=prefiller,
            tokenizer=tokenizer,
            generation_config=config['generation'],
            device=device
        )
        
        print(f"Free Generate Response: {free_generate_response}")
        print(f"Exit info: {exit_info}")
        
    except Exception as e:
        print(f"Free generate mode failed: {e}")

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])
Free Generate Response: <｜begin▁of▁sentence｜><｜Assistant｜> You are a helpful programming tutor.
<｜User｜> Explain the concept of recursion in programming.
<｜Assistant｜> 
Okay, so I need to explain how recursion works in programming. Hmm, I remember hearing about recursion before, but I'm not entirely understanding how it's different from other programming concepts. Let me try to break it down.

First, recursion is a method where a function calls itself. That means the function has a base case and/or recursive solution involving breaking into pieces. I think the base case is when you stop because you can't do any more work. But wait, how does that work in practice? Let's take example code.

I remember seeing something like this:

def factorial(n):
    if ...)
    return ...

But I'm unclear on the base case. Maybe it's when n is 0 or 1? Be